In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
from tensorflow.keras.layers import Dense

In [3]:
data=pd.read_csv("C:/Users/nchar/Desktop/english_to_french_translator/eng_-french.csv")
df=pd.DataFrame(data)
print(df.info())
df.dropna(axis=0,inplace=True)
english=df["English words/sentences"][0:10000]
french=df["French words/sentences"][0:10000]
english=list(french)
french=list(french)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
None


In [4]:
english=[(word.split()) for word in english]
french=[(word.split()) for word in french]
french

[['Salut!'],
 ['Cours', '!'],
 ['Courez', '!'],
 ['Qui', '?'],
 ['Ça', 'alors', '!'],
 ['Au', 'feu', '!'],
 ['À', "l'aide", '!'],
 ['Saute.'],
 ['Ça', 'suffit', '!'],
 ['Stop', '!'],
 ['Arrête-toi', '!'],
 ['Attends', '!'],
 ['Attendez', '!'],
 ['Poursuis.'],
 ['Continuez.'],
 ['Poursuivez.'],
 ['Bonjour', '!'],
 ['Salut', '!'],
 ['Je', 'comprends.'],
 ["J'essaye."],
 ["J'ai", 'gagné', '!'],
 ['Je', "l'ai", 'emporté', '!'],
 ['J’ai', 'gagné.'],
 ['Oh', 'non', '!'],
 ['Attaque', '!'],
 ['Attaquez', '!'],
 ['Santé', '!'],
 ['À', 'votre', 'santé', '!'],
 ['Merci', '!'],
 ['Tchin-tchin', '!'],
 ['Lève-toi.'],
 ['Va,', 'maintenant.'],
 ['Allez-y', 'maintenant.'],
 ['Vas-y', 'maintenant.'],
 ["J'ai", 'pigé', '!'],
 ['Compris', '!'],
 ['Pigé', '?'],
 ['Compris', '?'],
 ["T'as", 'capté', '?'],
 ['Monte.'],
 ['Montez.'],
 ['Serre-moi', 'dans', 'tes', 'bras', '!'],
 ['Serrez-moi', 'dans', 'vos', 'bras', '!'],
 ['Je', 'suis', 'tombée.'],
 ['Je', 'suis', 'tombé.'],
 ['Je', 'sais.'],
 ['Je', 'suis'

In [5]:
t1=tf.keras.preprocessing.text.Tokenizer()
t1.fit_on_texts(english)
t2=tf.keras.preprocessing.text.Tokenizer()
t2.fit_on_texts(french)

In [6]:
inp_encds_english=t1.texts_to_sequences(english)
inp_encds_french=t2.texts_to_sequences(french)

In [7]:
french_vocab_size=len(t2.word_index)
french_vocab_size

5525

In [8]:
english_max_len=max([len(text) for text in inp_encds_english])
french_max_len=max([len(text) for text in inp_encds_french])
def zero_padding_and_masking(sequences,max_len):
    batch_size=len(sequences)
    padded_sequences=np.zeros((batch_size,max_len),dtype=np.int32)
    mask=np.zeros((batch_size,max_len))
    for i,seq in enumerate(sequences):
        seq_len=len(seq)
        padded_sequences[i,:seq_len]=seq
        mask[i,:seq_len]=1
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=tf.reshape(mask,(batch_size,1,max_len)) # here we mask shape is (batch_size,seq_len) we reshape it to (batch_size,1,seq_len) or (100000,1,10)  because 
    # the shape of look_ahead_mask will be (batch_size,seq_len,seq_len) or (10000,10,10).hence reshape the padding_mask so that during addition of both array the padding mask will added perfectly with the help of broadcastting
    mask=tf.broadcast_to(mask,(10000,max_len,max_len))
    mask=tf.expand_dims(mask,axis=1) # here we expanded mask along the dimension 1 so it will be converted to (batch_size,1,seq_len,seq_len).we do this because when we add the mask to the qk/dk or attention weights the size of attention weights is (batch_size,num_heads,seq_len,seq_len

    return padded_sequences,mask
        
        
        
    
    
inp_encds_english,english_padding_mask=zero_padding_and_masking(inp_encds_english,english_max_len)
inp_encds_french,french_padding_mask=zero_padding_and_masking(inp_encds_french,french_max_len)
english_max_len,french_max_len

(10, 10)

In [9]:
inp_encds_english.shape,inp_encds_french.shape

((10000, 10), (10000, 10))

In [10]:
inp_encds_english

array([[2504,    0,    0, ...,    0,    0,    0],
       [1016,    2,    0, ...,    0,    0,    0],
       [1415,    2,    0, ...,    0,    0,    0],
       ...,
       [  38,   95,  413, ...,    0,    0,    0],
       [  38,   95, 1093, ...,    0,    0,    0],
       [  38,   95, 1094, ...,    0,    0,    0]])

In [11]:
inp_encds_french

array([[2504,    0,    0, ...,    0,    0,    0],
       [1016,    2,    0, ...,    0,    0,    0],
       [1415,    2,    0, ...,    0,    0,    0],
       ...,
       [  38,   95,  413, ...,    0,    0,    0],
       [  38,   95, 1093, ...,    0,    0,    0],
       [  38,   95, 1094, ...,    0,    0,    0]])

In [12]:
english_padding_mask.shape,french_padding_mask.shape

(TensorShape([10000, 1, 10, 10]), TensorShape([10000, 1, 10, 10]))

In [13]:
english_padding_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [14]:
embedding_dim=d_model=128
vocab_size=10000
english_embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
english_embedded_tokens=english_embedding_layer(inp_encds_english)
french_embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
french_embedded_tokens=french_embedding_layer(inp_encds_french)
print(english_embedded_tokens.shape)
french_embedded_tokens.shape

(10000, 10, 128)


TensorShape([10000, 10, 128])

In [15]:
num_heads=8
seq_len=10
dim_head=embedding_dim//num_heads
dim_head

16

In [16]:
def look_ahead_masking(seq_len,batch_size):
    mask=np.tril(np.ones((seq_len,seq_len)))
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=np.repeat(mask[np.newaxis,:,:],batch_size,axis=0)
    mask=tf.convert_to_tensor(mask)
    mask=tf.expand_dims(mask,axis=1)
    return mask


In [17]:
french_look_ahead_mask=look_ahead_masking(french_max_len,10000)
french_look_ahead_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
      

In [18]:
french_combined_mask=french_padding_mask+french_look_ahead_mask
french_combined_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [19]:
english_embedded_tokens

<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[ 0.04321017,  0.03776184,  0.01734996, ...,  0.03243711,
         -0.02772201,  0.04869641],
        [-0.03618106, -0.02113565, -0.03555471, ..., -0.00922464,
          0.03586209,  0.04355976],
        [-0.03618106, -0.02113565, -0.03555471, ..., -0.00922464,
          0.03586209,  0.04355976],
        ...,
        [-0.03618106, -0.02113565, -0.03555471, ..., -0.00922464,
          0.03586209,  0.04355976],
        [-0.03618106, -0.02113565, -0.03555471, ..., -0.00922464,
          0.03586209,  0.04355976],
        [-0.03618106, -0.02113565, -0.03555471, ..., -0.00922464,
          0.03586209,  0.04355976]],

       [[ 0.03688505, -0.03324645,  0.00364805, ..., -0.04478827,
          0.01181696, -0.02095742],
        [-0.00404375, -0.0070451 , -0.00385115, ..., -0.03389742,
         -0.04098063, -0.04387481],
        [-0.03618106, -0.02113565, -0.03555471, ..., -0.00922464,
          0.03586209,  0.04355976],
        .

In [20]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model:int,num_heads:int) -> None :
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.wq=Dense(d_model)
        self.wk=Dense(d_model)
        self.wv=Dense(d_model)
        self.dense=Dense(d_model)
        self.d_head=d_model//num_heads

    
    def scaled_dot_product_attention(self,q,k,v,mask):
        a=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        attention_weights=a/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_scores=tf.nn.softmax(attention_weights)
        outputs=tf.matmul(attention_scores,v)
        return outputs,attention_scores

    
    def split_heads(self, x, batch_size,seq_len):
        if tf.shape(x)[-1] % self.num_heads != 0:
            raise ValueError("The last dimension of the input tensor must be divisible by num_heads.")
    
        x = tf.reshape(x, (batch_size,seq_len, self.num_heads, self.d_head))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,x,mask):
        batch_size=tf.shape(x)[0]
        seq_len=tf.shape(x)[1]
        
        q=self.wq(x)
        k=self.wk(x)
        v=self.wv(x)
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        outputs,attention_weights=self.scaled_dot_product_attention(q,k,v,mask)
        attention = tf.transpose(outputs, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output

# Instantiate the MultiHeadAttention layer

# Example number of attention heads
mha_layer = MultiHeadAttention(d_model, num_heads)

# Call the layer
output = mha_layer(english_embedded_tokens,mask=english_padding_mask)
output

# Now `output` will be the result of the multi-head attention operation


<tf.Tensor: shape=(10000, 10, 128), dtype=float32, numpy=
array([[[-0.01375226,  0.00356941, -0.01974104, ...,  0.02305207,
         -0.03901201, -0.06490552],
        [-0.01375226,  0.00356941, -0.01974104, ...,  0.02305207,
         -0.03901201, -0.06490552],
        [-0.01375226,  0.00356941, -0.01974104, ...,  0.02305207,
         -0.03901201, -0.06490552],
        ...,
        [-0.01375226,  0.00356941, -0.01974104, ...,  0.02305207,
         -0.03901201, -0.06490552],
        [-0.01375226,  0.00356941, -0.01974104, ...,  0.02305207,
         -0.03901201, -0.06490552],
        [-0.01375226,  0.00356941, -0.01974104, ...,  0.02305207,
         -0.03901201, -0.06490552]],

       [[ 0.01258055,  0.00829556,  0.0010081 , ...,  0.01921292,
          0.01462522, -0.02755656],
        [ 0.0125944 ,  0.00829711,  0.00100132, ...,  0.01920782,
          0.01463027, -0.02754479],
        [ 0.0126043 ,  0.0082922 ,  0.00102177, ...,  0.01918443,
          0.01464668, -0.02751708],
        .

In [21]:
class Feedforward(tf.keras.layers.Layer):
    def __init__(self,d_model,dff):
        super().__init__()
        self.d_model=d_model
        self.dff=dff
        self.layer1=tf.keras.layers.Dense(dff,activation="relu")
        self.layer2=tf.keras.layers.Dense(d_model)
    def call(self,x):
        out1=self.layer1(x)
        out2=self.layer2(out1)
        return out2

In [22]:
class CrossMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads):
        super().__init__()
        self.num_heads=num_heads
        self.d_model=d_model
        self.wq=tf.keras.layers.Dense(d_model)
        self.wk=tf.keras.layers.Dense(d_model)
        self.wv=tf.keras.layers.Dense(d_model)
        self.dense=tf.keras.layers.Dense(d_model)
        self.d_head=d_model//num_heads


    def split_heads(self,y,batch_size,seq_len):
        y=tf.reshape(y,(batch_size,seq_len,self.num_heads,self.d_head))
        y=tf.transpose(y,perm=[0,2,1,3])
        return y


    def scaled_dot_product_attention(self,q,k,v,mask):
        qk=tf.matmul(q,tf.transpose(k,perm=[0,1,3,2]))
        dk=float(tf.shape(k)[-1])
        attention_weights=qk/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_score=tf.nn.softmax(attention_weights)
        output=tf.matmul(attention_score,v)
        return output


    def call(self,x,y,mask):
        q=self.wq(y)
        k=self.wk(x)
        v=self.wv(x)
        batch_size=tf.shape(q)[0]
        seq_len=tf.shape(q)[1]
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        attention=self.scaled_dot_product_attention(q,k,v,mask)
        out=self.dense(attention)
        ouput=tf.transpose(out,perm=[0,2,1,3])
        final_output=tf.reshape(output,(batch_size,seq_len,self.d_model))
        return final_output
 

In [23]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
    def call(self,x,mask):
        attention_out=self.attention(x,mask)
        norm_and_add1_out=self.norm_and_add1(x+attention_out)
        feed_forward_out=self.feed_forward(norm_and_add1_out)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+feed_forward_out)
        return norm_and_add2_out

In [24]:
class SequentialEncoder(tf.keras.layers.Layer):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def call(self, x,mask):
        for layer in self.layers:
            x = layer(x,mask)
        return x

In [25]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model,num_heads,dff,num_encoders):
        super().__init__()
        self.layers = SequentialEncoder([Encoder_layer(d_model=d_model,num_heads=num_heads,dff=dff) for _ in range(num_encoders)])

    def call(self, x,mask):
        x = self.layers(x,mask)
        return x

In [26]:
class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.dff=dff
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.encoder_decoder_attention=CrossMultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
        self.norm_and_add3=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    def call(self,x,y,combined_mask,padding_mask): #x==>encoder's final query vector(output)
        attention_out=self.attention(y,mask=combined_mask)
        norm_and_add1_out=self.norm_and_add1(attention_out+y)
        encoder_decoder_attention_out=self.encoder_decoder_attention(x,y,mask=padding_mask)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+encoder_decoder_attention_out)
        feed_forward_out=self.feed_forward(norm_and_add2_out)
        norm_and_add3_out=self.norm_and_add3(norm_and_add2_out+feed_forward_out)
        return norm_and_add3_out    

In [27]:
class SequentialDecoder(tf.keras.layers.Layer):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def call(self, x,y,combined_mask,padding_mask):
        for layer in self.layers:
            x = layer(x,y,combined_mask,padding_mask)
        return x

In [28]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff,num_decoders):
        super().__init__()
        self.num_decoders=num_decoders
        self.layers=SequentialDecoder([Decoder_layer(d_model=d_model,num_heads=num_heads,dff=dff) for _ in range(num_decoders)])
    def call(self,x,y,combined_mask,padding_mask):
        y=self.layers(x,y,combined_mask,padding_mask)
        return y

In [29]:
class Transformer(tf.keras.Model):
    def __init__(self,d_model,num_heads,dff,num_layers,target_vocab_size):
        super().__init__()
        self.encoder=Encoder(d_model=d_model,num_heads=num_heads,dff=dff,num_encoders=num_layers)
        self.decoder=Decoder(d_model=d_model,num_heads=num_heads,dff=dff,num_decoders=num_layers)
        self.final_layer=tf.keras.layers.Dense(target_vocab_size)
    def call(self,input,output,input_padding_mask,output_padding_mask,output_look_ahead_mask):
        input=self.encoder(input,input_padding_mask)
        decoder_out=self.decoder(input,output,output_look_ahead_mask,output_padding_mask)
        out=self.final_layer(decoder_out)
        return out
        

In [30]:
transformer=Transformer(d_model,num_heads=8,dff=512,num_layers=8,target_vocab_size=french_vocab_size)
transformer

<Transformer name=transformer, built=False>

In [31]:
out=transformer(english_embedded_tokens,french_embedded_tokens,english_padding_mask,french_padding_mask,french_look_ahead_mask)
out

<tf.Tensor: shape=(10000, 10, 5525), dtype=float32, numpy=
array([[[ 0.11069676, -0.06885329, -0.09797797, ..., -0.03225675,
         -0.08658363, -0.22106539],
        [ 0.09364279,  0.20192592, -0.15352449, ..., -0.36096257,
         -0.06266884, -0.02222506],
        [ 0.08973873,  0.22157887, -0.14851408, ..., -0.37018538,
         -0.06442969,  0.02367582],
        ...,
        [ 0.09324589,  0.24376109, -0.15170442, ..., -0.3627024 ,
         -0.08066013,  0.07504055],
        [ 0.09361357,  0.24526872, -0.15139315, ..., -0.36193818,
         -0.08146777,  0.07762035],
        [ 0.09388412,  0.24638692, -0.1511646 , ..., -0.36131477,
         -0.08213075,  0.07955946]],

       [[ 0.24265672, -0.02474189, -0.19073357, ..., -0.24916242,
         -0.1695719 ,  0.06906618],
        [ 0.14715081,  0.28562772, -0.2115316 , ..., -0.08052523,
         -0.24907646,  0.09562156],
        [ 0.1492957 ,  0.23779128, -0.22273567, ..., -0.56051093,
         -0.2370122 ,  0.2040539 ],
        

In [38]:
transformer.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_4/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_4/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_5/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_5/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_6/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_6/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_

In [33]:
out_probabilities = tf.nn.softmax(out, axis=-1)
out

<tf.Tensor: shape=(10000, 10, 5525), dtype=float32, numpy=
array([[[ 0.11069676, -0.06885329, -0.09797797, ..., -0.03225675,
         -0.08658363, -0.22106539],
        [ 0.09364279,  0.20192592, -0.15352449, ..., -0.36096257,
         -0.06266884, -0.02222506],
        [ 0.08973873,  0.22157887, -0.14851408, ..., -0.37018538,
         -0.06442969,  0.02367582],
        ...,
        [ 0.09324589,  0.24376109, -0.15170442, ..., -0.3627024 ,
         -0.08066013,  0.07504055],
        [ 0.09361357,  0.24526872, -0.15139315, ..., -0.36193818,
         -0.08146777,  0.07762035],
        [ 0.09388412,  0.24638692, -0.1511646 , ..., -0.36131477,
         -0.08213075,  0.07955946]],

       [[ 0.24265672, -0.02474189, -0.19073357, ..., -0.24916242,
         -0.1695719 ,  0.06906618],
        [ 0.14715081,  0.28562772, -0.2115316 , ..., -0.08052523,
         -0.24907646,  0.09562156],
        [ 0.1492957 ,  0.23779128, -0.22273567, ..., -0.56051093,
         -0.2370122 ,  0.2040539 ],
        

In [34]:
transformer.encoder.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_4/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_4/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_5/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_5/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_6/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_6/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_

In [40]:
len(transformer.encoder.trainable_variables)

128

In [35]:
transformer.decoder.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/dense_52/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/dense_52/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/dense_53/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/dense_53/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/dense_54/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/dense_54/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/decoder/sequential_decoder/decoder_layer/multi_head_attention_9/

In [41]:
len(transformer.decoder.trainable_variables)

208

In [39]:
transformer.trainable_variables

[<KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_4/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_4/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_5/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_5/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_6/kernel>,
 <KerasVariable shape=(128,), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_6/bias>,
 <KerasVariable shape=(128, 128), dtype=float32, path=transformer/encoder/sequential_encoder/encoder_layer/multi_head_attention_1/dense_

In [42]:
len(transformer.trainable_variables)

338

In [43]:
128+208

336